# Team Project - OCDM

## Explanation 
In this document is the code written to scrape the data from vivino, to look at if price have influence on the ratings of the different wines.

### Code
We will use selenium and Beautifull Soup in this document

#### Install the packages & librabies 

Explanation what we are doing:

In [107]:
#installing packages

!pip3 install selenium
!pip3 install webdriver_manager

#libraries

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

# import requests
# import from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup
import time

## Red wines 

In the cells below we are gonna collect the data of the red wines. To be safe that the code don´t break, we seperate the code into two benchmarks where we first collect the data of the red wines in the price range between 0-20 euro´s.


### Red wines in the price range 0-10

In [108]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Opening the 'Vivino' website
url = "https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1NFBLrrT181FLtnUNDVIrAEqmp9mWJRZlppYk5qjlF6XYpqQWJ6vlJ1XaFhRlJqeqlZdExwJVJVcWA-nUYjUwCQC3hRy5"
driver.get(url)

# Optional: Adding some wait time for the page to fully load if needed
driver.implicitly_wait(10)  # 10 seconds
time.sleep(20)

In [109]:
try:
    # Create a function to click away the cookies
    accept_cookies_button = driver.find_element(By.ID,"didomi-notice-agree-button")
    accept_cookies_button.click()
    print("Cookies accepted.")
except Exception as e:
    print("Cookie acceptance button not found or could not be clicked:", e)


Cookies accepted.


The code below provides to taggle the infinity scroll, this takes a long time like (15 minutes) till the end of the infitity scroll.

In [110]:
from bs4 import BeautifulSoup
import time
driver.implicitly_wait(10)

# Infinite scroll to load more content
scroll_pause_time = 5  # Adjust if necessary
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for the new page to load
    time.sleep(scroll_pause_time)
    
    # Calculate new scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # Stop if we've reached the end of the page
    last_height = new_height

# Get the final page source after all content is loaded
page_source = driver.page_source

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

Scraping all the data

In [111]:
import csv
from bs4 import BeautifulSoup

# Create empty lists to store the data
brands = []
wines = []
ratings = []
prices = []

# Find all wine entries on the page
wine_entries = soup.find_all(class_='card__card--2R5Wh wineCard__wineCardContent--3cwZt')

for entry in wine_entries:
    # Extract brand
    brand = entry.find(class_='wineInfoVintage__truncate--3QAtw')
    if brand:
        brands.append(brand.get_text(strip=True))

    # Extract wine name
    wine = entry.find(class_='wineInfoVintage__vintage--VvWlU wineInfoVintage__truncate--3QAtw') 
    if wine:
        wines.append(wine.get_text(strip=True))

    # Extract rating
    rating = entry.find(class_='vivinoRating_averageValue__uDdPM')
    if rating:
        ratings.append(rating.get_text(strip=True))

# Check for the presence of the discount first
    discount_price_div = entry.find(class_='price_strike__mOVjZ addToCart__subText--1pvFt')
    if discount_price_div:
        # If discount exist, get the original price
        discount_price_text = discount_price_div.get_text(strip=True)
        price_only = discount_price_text.split()[-1]  # Get the last part (currency + price)
        prices.append(price_only)  # Append only the discount price to the list
    
    else:  
        # Extract currency & price if present in the addToCartButton
        price_divs = entry.find_all(class_='addToCartButton__price--qJdh4')
        if price_divs:  # If primary price class exists
            for price_div in price_divs:
                currency = price_div.find('div', class_='addToCartButton__currency--2CTNX')
                price = price_div.find_all('div')[1]  # Assuming price is in the second div
                full_price = f"{currency.get_text(strip=True) if currency else ''}{price.get_text(strip=True) if price else ''}"
                prices.append(full_price)  # Save the full price
        else:  # If not present, extract price from alternative class (online verkrijgbaar vanaf...)
            alt_price_div = entry.find(class_='addToCart__subText--1pvFt addToCart__ppcPrice--ydrd5')
            if alt_price_div:
                alt_price_text = alt_price_div.get_text(strip=True)
                price_only = alt_price_text.split()[-1]  # Get the last part (currency + price)
                prices.append(price_only)  # Append only the price to the list

Saving the data to a CSV file

In [112]:
with open('red_wine_0_10.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['Brand', 'Wine', 'Rating', 'Price'])

    # Write the data rows
    for brand, wine, rating, price in zip(brands, wines, ratings, prices):
        writer.writerow([
            brand,
            wine,
            rating,
            price
        ])

print("Data saved to 'red_wine_0_10.csv' successfully.")


Data saved to 'red_wine_0_10.csv' successfully.


### Red Wine within price range 10-20 euro

In [113]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Opening the 'Vivino' website
url = 'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1NFDLTaywNTJQS6609fNRS7Z1DQ1SKwDKpqfZliUWZaaWJOao5Rel2KakFier5SdV2hYUZSanqpWXRMcCVSVXFgPp1GI1MAkAya8c6w%3D%3D'
driver.get(url)

# Optional: Adding some wait time for the page to fully load if needed
driver.implicitly_wait(10)  # 10 seconds
time.sleep(20)


In [114]:
try:
    # Create a function to click away the cookies
    accept_cookies_button = driver.find_element(By.ID,"didomi-notice-agree-button")
    accept_cookies_button.click()
    print("Cookies accepted.")
except Exception as e:
    print("Cookie acceptance button not found or could not be clicked:", e)

Cookies accepted.


In [115]:
from bs4 import BeautifulSoup
import time
driver.implicitly_wait(30)

# Infinite scroll to load more content
scroll_pause_time = 5  # Adjust if necessary
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for the new page to load
    time.sleep(scroll_pause_time)
    
    # Calculate new scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # Stop if we've reached the end of the page
    last_height = new_height

# Get the final page source after all content is loaded
page_source = driver.page_source

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

In [116]:
import csv
from bs4 import BeautifulSoup

# Create empty lists to store the data
brands = []
wines = []
ratings = []
prices = []

# Find all wine entries on the page
wine_entries = soup.find_all(class_='card__card--2R5Wh wineCard__wineCardContent--3cwZt')

for entry in wine_entries:
    # Extract brand
    brand = entry.find(class_='wineInfoVintage__truncate--3QAtw')
    if brand:
        brands.append(brand.get_text(strip=True))

    # Extract wine name
    wine = entry.find(class_='wineInfoVintage__vintage--VvWlU wineInfoVintage__truncate--3QAtw') 
    if wine:
        wines.append(wine.get_text(strip=True))

    # Extract rating
    rating = entry.find(class_='vivinoRating_averageValue__uDdPM')
    if rating:
        ratings.append(rating.get_text(strip=True))

# Check for the presence of the discount first
    discount_price_div = entry.find(class_='price_strike__mOVjZ addToCart__subText--1pvFt')
    if discount_price_div:
        # If discount exist, get the original price
        discount_price_text = discount_price_div.get_text(strip=True)
        price_only = discount_price_text.split()[-1]  # Get the last part (currency + price)
        prices.append(price_only)  # Append only the discount price to the list
    
    else:  
        # Extract currency & price if present in the addToCartButton
        price_divs = entry.find_all(class_='addToCartButton__price--qJdh4')
        if price_divs:  # If primary price class exists
            for price_div in price_divs:
                currency = price_div.find('div', class_='addToCartButton__currency--2CTNX')
                price = price_div.find_all('div')[1]  # Assuming price is in the second div
                full_price = f"{currency.get_text(strip=True) if currency else ''}{price.get_text(strip=True) if price else ''}"
                prices.append(full_price)  # Save the full price
        else:  # If not present, extract price from alternative class (online verkrijgbaar vanaf...)
            alt_price_div = entry.find(class_='addToCart__subText--1pvFt addToCart__ppcPrice--ydrd5')
            if alt_price_div:
                alt_price_text = alt_price_div.get_text(strip=True)
                price_only = alt_price_text.split()[-1]  # Get the last part (currency + price)
                prices.append(price_only)  # Append only the price to the list

In [117]:
with open('red_wine_10_20.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['Brand', 'Wine', 'Rating', 'Price'])

    # Write the data rows
    for brand, wine, rating, price in zip(brands, wines, ratings, prices):
        writer.writerow([
            brand,
            wine,
            rating,
            price
        ])

print("Data saved to 'red_wine_10_20.csv' successfully.")


Data saved to 'red_wine_10_20.csv' successfully.


### Red Wine within price range 20-40 euro

In [118]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Opening the 'Vivino' website
url = 'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1MlDLTaywNTFQS6609fNRS7Z1DQ1SKwDKpqfZliUWZaaWJOao5Rel2KakFier5SdV2hYUZSanqpWXRMcCVSVXFgPp1GI1MAkAyosc7g%3D%3D'
driver.get(url)

# Optional: Adding some wait time for the page to fully load if needed
driver.implicitly_wait(10)  # 10 seconds
time.sleep(20)


In [119]:
try:
    # Create a function to click away the cookies
    accept_cookies_button = driver.find_element(By.ID,"didomi-notice-agree-button")
    accept_cookies_button.click()
    print("Cookies accepted.")
except Exception as e:
    print("Cookie acceptance button not found or could not be clicked:", e)

Cookies accepted.


In [120]:
from bs4 import BeautifulSoup
import time
driver.implicitly_wait(30)

# Infinite scroll to load more content
scroll_pause_time = 5  # Adjust if necessary
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for the new page to load
    time.sleep(scroll_pause_time)
    
    # Calculate new scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # Stop if we've reached the end of the page
    last_height = new_height

# Get the final page source after all content is loaded
page_source = driver.page_source

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

In [121]:
import csv
from bs4 import BeautifulSoup

# Create empty lists to store the data
brands = []
wines = []
ratings = []
prices = []

# Find all wine entries on the page
wine_entries = soup.find_all(class_='card__card--2R5Wh wineCard__wineCardContent--3cwZt')

for entry in wine_entries:
    # Extract brand
    brand = entry.find(class_='wineInfoVintage__truncate--3QAtw')
    if brand:
        brands.append(brand.get_text(strip=True))

    # Extract wine name
    wine = entry.find(class_='wineInfoVintage__vintage--VvWlU wineInfoVintage__truncate--3QAtw') 
    if wine:
        wines.append(wine.get_text(strip=True))

    # Extract rating
    rating = entry.find(class_='vivinoRating_averageValue__uDdPM')
    if rating:
        ratings.append(rating.get_text(strip=True))

# Check for the presence of the discount first
    discount_price_div = entry.find(class_='price_strike__mOVjZ addToCart__subText--1pvFt')
    if discount_price_div:
        # If discount exist, get the original price
        discount_price_text = discount_price_div.get_text(strip=True)
        price_only = discount_price_text.split()[-1]  # Get the last part (currency + price)
        prices.append(price_only)  # Append only the discount price to the list
    
    else:  
        # Extract currency & price if present in the addToCartButton
        price_divs = entry.find_all(class_='addToCartButton__price--qJdh4')
        if price_divs:  # If primary price class exists
            for price_div in price_divs:
                currency = price_div.find('div', class_='addToCartButton__currency--2CTNX')
                price = price_div.find_all('div')[1]  # Assuming price is in the second div
                full_price = f"{currency.get_text(strip=True) if currency else ''}{price.get_text(strip=True) if price else ''}"
                prices.append(full_price)  # Save the full price
        else:  # If not present, extract price from alternative class (online verkrijgbaar vanaf...)
            alt_price_div = entry.find(class_='addToCart__subText--1pvFt addToCart__ppcPrice--ydrd5')
            if alt_price_div:
                alt_price_text = alt_price_div.get_text(strip=True)
                price_only = alt_price_text.split()[-1]  # Get the last part (currency + price)
                prices.append(price_only)  # Append only the price to the list

In [122]:
with open('red_wine_20_40.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['Brand', 'Wine', 'Rating', 'Price'])

    # Write the data rows
    for brand, wine, rating, price in zip(brands, wines, ratings, prices):
        writer.writerow([
            brand,
            wine,
            rating,
            price
        ])

print("Data saved to 'red_wine_20_40.csv' successfully.")


Data saved to 'red_wine_20_40.csv' successfully.


### Red Wine within price range 40-500 euro

In [123]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Opening the 'Vivino' website
url = 'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1MVDLTaywNTUwUEuutPXzUUu2dQ0NUisASqen2ZYlFmWmliTmqOUXpdimpBYnq-UnVdoWFGUmp6qVl0THAlUlVxYD6dRiNTAJAN31HSE%3D'
driver.get(url)

# Optional: Adding some wait time for the page to fully load if needed
driver.implicitly_wait(10)  # 10 seconds
time.sleep(20)


In [124]:
try:
    # Create a function to click away the cookies
    accept_cookies_button = driver.find_element(By.ID,"didomi-notice-agree-button")
    accept_cookies_button.click()
    print("Cookies accepted.")
except Exception as e:
    print("Cookie acceptance button not found or could not be clicked:", e)

Cookies accepted.


In [125]:
from bs4 import BeautifulSoup
import time
driver.implicitly_wait(30)

# Infinite scroll to load more content
scroll_pause_time = 5  # Adjust if necessary
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for the new page to load
    time.sleep(scroll_pause_time)
    
    # Calculate new scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # Stop if we've reached the end of the page
    last_height = new_height

# Get the final page source after all content is loaded
page_source = driver.page_source

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

In [126]:
import csv
from bs4 import BeautifulSoup

# Create empty lists to store the data
brands = []
wines = []
ratings = []
prices = []

# Find all wine entries on the page
wine_entries = soup.find_all(class_='card__card--2R5Wh wineCard__wineCardContent--3cwZt')

for entry in wine_entries:
    # Extract brand
    brand = entry.find(class_='wineInfoVintage__truncate--3QAtw')
    if brand:
        brands.append(brand.get_text(strip=True))

    # Extract wine name
    wine = entry.find(class_='wineInfoVintage__vintage--VvWlU wineInfoVintage__truncate--3QAtw') 
    if wine:
        wines.append(wine.get_text(strip=True))

    # Extract rating
    rating = entry.find(class_='vivinoRating_averageValue__uDdPM')
    if rating:
        ratings.append(rating.get_text(strip=True))

# Check for the presence of the discount first
    discount_price_div = entry.find(class_='price_strike__mOVjZ addToCart__subText--1pvFt')
    if discount_price_div:
        # If discount exist, get the original price
        discount_price_text = discount_price_div.get_text(strip=True)
        price_only = discount_price_text.split()[-1]  # Get the last part (currency + price)
        prices.append(price_only)  # Append only the discount price to the list
    
    else:  
        # Extract currency & price if present in the addToCartButton
        price_divs = entry.find_all(class_='addToCartButton__price--qJdh4')
        if price_divs:  # If primary price class exists
            for price_div in price_divs:
                currency = price_div.find('div', class_='addToCartButton__currency--2CTNX')
                price = price_div.find_all('div')[1]  # Assuming price is in the second div
                full_price = f"{currency.get_text(strip=True) if currency else ''}{price.get_text(strip=True) if price else ''}"
                prices.append(full_price)  # Save the full price
        else:  # If not present, extract price from alternative class (online verkrijgbaar vanaf...)
            alt_price_div = entry.find(class_='addToCart__subText--1pvFt addToCart__ppcPrice--ydrd5')
            if alt_price_div:
                alt_price_text = alt_price_div.get_text(strip=True)
                price_only = alt_price_text.split()[-1]  # Get the last part (currency + price)
                prices.append(price_only)  # Append only the price to the list

In [127]:
with open('red_wine_40_500.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['Brand', 'Wine', 'Rating', 'Price'])

    # Write the data rows
    for brand, wine, rating, price in zip(brands, wines, ratings, prices):
        writer.writerow([
            brand,
            wine,
            rating,
            price
        ])

print("Data saved to 'red_wine_40_500.csv' successfully.")


Data saved to 'red_wine_40_500.csv' successfully.


## White Wines
In the cells below, we are gonna collect the data of the white wines. To be safe that the code will not break and that we have to wait long to collect the data, we have created benchmark.

### White wines within price range 0-20 euros.

In [128]:
## Witte Spaanse Wijnen 0-200 euro


service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Opening the 'Vivino' website
url = 'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1MlBLrrT181FLtnUNDVIrAEqmp9mWJRZlppYk5qjlF6XYJhYnq-UnVdoWFGUmp6qVl0TH2hoBNRUD6dRiNTAJAJlRHFM%3D'
driver.get(url)

# Optional: Adding some wait time for the page to fully load if needed

driver.implicitly_wait(10)  # 10 seconds 
time.sleep(20)

In [129]:
try:
    # Create a function to click away the cookies
    accept_cookies_button = driver.find_element(By.ID,"didomi-notice-agree-button")
    accept_cookies_button.click()
    print("Cookies accepted.")
except Exception as e:
    print("Cookie acceptance button not found or could not be clicked:", e)


Cookies accepted.


In [130]:
from bs4 import BeautifulSoup
import time
driver.implicitly_wait(10)

# Infinite scroll to load more content
scroll_pause_time = 5  # Adjust if necessary
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for the new page to load
    time.sleep(scroll_pause_time)
    
    # Calculate new scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # Stop if we've reached the end of the page
    last_height = new_height

# Get the final page source after all content is loaded
page_source = driver.page_source

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

In [131]:
import csv
from bs4 import BeautifulSoup

# Create empty lists to store the data
brands = []
wines = []
ratings = []
prices = []

# Find all wine entries on the page
wine_entries = soup.find_all(class_='card__card--2R5Wh wineCard__wineCardContent--3cwZt')

for entry in wine_entries:
    # Extract brand
    brand = entry.find(class_='wineInfoVintage__truncate--3QAtw')
    if brand:
        brands.append(brand.get_text(strip=True))

    # Extract wine name
    wine = entry.find(class_='wineInfoVintage__vintage--VvWlU wineInfoVintage__truncate--3QAtw') 
    if wine:
        wines.append(wine.get_text(strip=True))

    # Extract rating
    rating = entry.find(class_='vivinoRating_averageValue__uDdPM')
    if rating:
        ratings.append(rating.get_text(strip=True))

# Check for the presence of the discount first
    discount_price_div = entry.find(class_='price_strike__mOVjZ addToCart__subText--1pvFt')
    if discount_price_div:
        # If discount exist, get the original price
        discount_price_text = discount_price_div.get_text(strip=True)
        price_only = discount_price_text.split()[-1]  # Get the last part (currency + price)
        prices.append(price_only)  # Append only the discount price to the list
    
    else:  
        # Extract currency & price if present in the addToCartButton
        price_divs = entry.find_all(class_='addToCartButton__price--qJdh4')
        if price_divs:  # If primary price class exists
            for price_div in price_divs:
                currency = price_div.find('div', class_='addToCartButton__currency--2CTNX')
                price = price_div.find_all('div')[1]  # Assuming price is in the second div
                full_price = f"{currency.get_text(strip=True) if currency else ''}{price.get_text(strip=True) if price else ''}"
                prices.append(full_price)  # Save the full price
        else:  # If not present, extract price from alternative class (online verkrijgbaar vanaf...)
            alt_price_div = entry.find(class_='addToCart__subText--1pvFt addToCart__ppcPrice--ydrd5')
            if alt_price_div:
                alt_price_text = alt_price_div.get_text(strip=True)
                price_only = alt_price_text.split()[-1]  # Get the last part (currency + price)
                prices.append(price_only)  # Append only the price to the list

In [132]:
with open('white_wine_0_20.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['Brand', 'Wine', 'Rating', 'Price'])

    # Write the data rows
    for brand, wine, rating, price in zip(brands, wines, ratings, prices):
        writer.writerow([
            brand,
            wine,
            rating,
            price
        ])

print("Data saved to 'white_wine_0_20.csv' successfully.")


Data saved to 'white_wine_0_20.csv' successfully.


### White wine within the price range 20-500 euro

In [133]:
## Witte Spaanse Wijnen 21-500 euro

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Opening the 'Vivino' website
url = 'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1MlDLTaywNTUwUEuutPXzUUu2dQ0NUisASqen2ZYlFmWmliTmqOUXpdgmFier5SdV2hYUZSanqpWXRMfaGgE1FQPp1GI1MAkAvnccuA%3D%3D'
driver.get(url)

# Optional: Adding some wait time for the page to fully load if needed

driver.implicitly_wait(10)  # 10 seconds 
time.sleep(20)

In [134]:
try:
    # Create a function to click away the cookies
    accept_cookies_button = driver.find_element(By.ID,"didomi-notice-agree-button")
    accept_cookies_button.click()
    print("Cookies accepted.")
except Exception as e:
    print("Cookie acceptance button not found or could not be clicked:", e)
    

Cookies accepted.


In [135]:
from bs4 import BeautifulSoup
import time
driver.implicitly_wait(10)

# Infinite scroll to load more content
scroll_pause_time = 5  # Adjust if necessary
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for the new page to load
    time.sleep(scroll_pause_time)
    
    # Calculate new scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # Stop if we've reached the end of the page
    last_height = new_height

# Get the final page source after all content is loaded
page_source = driver.page_source

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

In [136]:
import csv
from bs4 import BeautifulSoup

# Create empty lists to store the data
brands = []
wines = []
ratings = []
prices = []

# Find all wine entries on the page
wine_entries = soup.find_all(class_='card__card--2R5Wh wineCard__wineCardContent--3cwZt')

for entry in wine_entries:
    # Extract brand
    brand = entry.find(class_='wineInfoVintage__truncate--3QAtw')
    if brand:
        brands.append(brand.get_text(strip=True))

    # Extract wine name
    wine = entry.find(class_='wineInfoVintage__vintage--VvWlU wineInfoVintage__truncate--3QAtw') 
    if wine:
        wines.append(wine.get_text(strip=True))

    # Extract rating
    rating = entry.find(class_='vivinoRating_averageValue__uDdPM')
    if rating:
        ratings.append(rating.get_text(strip=True))

# Check for the presence of the discount first
    discount_price_div = entry.find(class_='price_strike__mOVjZ addToCart__subText--1pvFt')
    if discount_price_div:
        # If discount exist, get the original price
        discount_price_text = discount_price_div.get_text(strip=True)
        price_only = discount_price_text.split()[-1]  # Get the last part (currency + price)
        prices.append(price_only)  # Append only the discount price to the list
    
    else:  
        # Extract currency & price if present in the addToCartButton
        price_divs = entry.find_all(class_='addToCartButton__price--qJdh4')
        if price_divs:  # If primary price class exists
            for price_div in price_divs:
                currency = price_div.find('div', class_='addToCartButton__currency--2CTNX')
                price = price_div.find_all('div')[1]  # Assuming price is in the second div
                full_price = f"{currency.get_text(strip=True) if currency else ''}{price.get_text(strip=True) if price else ''}"
                prices.append(full_price)  # Save the full price
        else:  # If not present, extract price from alternative class (online verkrijgbaar vanaf...)
            alt_price_div = entry.find(class_='addToCart__subText--1pvFt addToCart__ppcPrice--ydrd5')
            if alt_price_div:
                alt_price_text = alt_price_div.get_text(strip=True)
                price_only = alt_price_text.split()[-1]  # Get the last part (currency + price)
                prices.append(price_only)  # Append only the price to the list

In [137]:
with open('white_wine_20_500.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['Brand', 'Wine', 'Rating', 'Price'])

    # Write the data rows
    for brand, wine, rating, price in zip(brands, wines, ratings, prices):
        writer.writerow([
            brand,
            wine,
            rating,
            price
        ])

print("Data saved to 'white_wine_20_500.csv' successfully.")

Data saved to 'white_wine_20_500.csv' successfully.


## Rose Wine
some text

In [138]:
#Rose wines 0-500

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Opening the 'Vivino' website
url = 'https://www.vivino.com/explore?e=eJwdi70KgCAABt_mmw1q_Ma2aAiaIsLMQkgNtb-3T1ruhuNsYAFrHAWsfFgJAfWybaBY9x2OXLeVlwxGJ7nDh4WLjgp-fhlkMm6Lk_KnS7jTMLLMd8zWET8_0d4gXQ%3D%3D'
driver.get(url)

# Optional: Adding some wait time for the page to fully load if needed

driver.implicitly_wait(10)  # 10 seconds 
time.sleep(20)

In [139]:
try:
    # Create a function to click away the cookies
    accept_cookies_button = driver.find_element(By.ID,"didomi-notice-agree-button")
    accept_cookies_button.click()
    print("Cookies accepted.")
except Exception as e:
    print("Cookie acceptance button not found or could not be clicked:", e)

Cookies accepted.


In [140]:
from bs4 import BeautifulSoup
import time
driver.implicitly_wait(10)

# Infinite scroll to load more content
scroll_pause_time = 5  # Adjust if necessary
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for the new page to load
    time.sleep(scroll_pause_time)
    
    # Calculate new scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # Stop if we've reached the end of the page
    last_height = new_height

# Get the final page source after all content is loaded
page_source = driver.page_source

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

In [141]:
import csv
from bs4 import BeautifulSoup

# Create empty lists to store the data
brands = []
wines = []
ratings = []
prices = []

# Find all wine entries on the page
wine_entries = soup.find_all(class_='card__card--2R5Wh wineCard__wineCardContent--3cwZt')

for entry in wine_entries:
    # Extract brand
    brand = entry.find(class_='wineInfoVintage__truncate--3QAtw')
    if brand:
        brands.append(brand.get_text(strip=True))

    # Extract wine name
    wine = entry.find(class_='wineInfoVintage__vintage--VvWlU wineInfoVintage__truncate--3QAtw') 
    if wine:
        wines.append(wine.get_text(strip=True))

    # Extract rating
    rating = entry.find(class_='vivinoRating_averageValue__uDdPM')
    if rating:
        ratings.append(rating.get_text(strip=True))

# Check for the presence of the discount first
    discount_price_div = entry.find(class_='price_strike__mOVjZ addToCart__subText--1pvFt')
    if discount_price_div:
        # If discount exist, get the original price
        discount_price_text = discount_price_div.get_text(strip=True)
        price_only = discount_price_text.split()[-1]  # Get the last part (currency + price)
        prices.append(price_only)  # Append only the discount price to the list
    
    else:  
        # Extract currency & price if present in the addToCartButton
        price_divs = entry.find_all(class_='addToCartButton__price--qJdh4')
        if price_divs:  # If primary price class exists
            for price_div in price_divs:
                currency = price_div.find('div', class_='addToCartButton__currency--2CTNX')
                price = price_div.find_all('div')[1]  # Assuming price is in the second div
                full_price = f"{currency.get_text(strip=True) if currency else ''}{price.get_text(strip=True) if price else ''}"
                prices.append(full_price)  # Save the full price
        else:  # If not present, extract price from alternative class (online verkrijgbaar vanaf...)
            alt_price_div = entry.find(class_='addToCart__subText--1pvFt addToCart__ppcPrice--ydrd5')
            if alt_price_div:
                alt_price_text = alt_price_div.get_text(strip=True)
                price_only = alt_price_text.split()[-1]  # Get the last part (currency + price)
                prices.append(price_only)  # Append only the price to the list

In [142]:
with open('rose_wine.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['Brand', 'Wine', 'Rating', 'Price'])

    # Write the data rows
    for brand, wine, rating, price in zip(brands, wines, ratings, prices):
        writer.writerow([
            brand,
            wine,
            rating,
            price
        ])

print("Data saved to 'rose_wine.csv' successfully.")

Data saved to 'rose_wine.csv' successfully.


## Sparkling Wine
Nog aanpassen

In [143]:
## Witte Spaanse Wijnen 21-500 euro

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Opening the 'Vivino' website
url = 'https://www.vivino.com/explore?e=eJwdi70KgCAABt_mm41o_Ma2aAiaIsLMQkgNtb-3T1ruhuNsYAFrHAWsfFgJAfWybaBY9x2OXLeVlwxGJ7nDh4WLjgp-fhlkMm6Lk_KnS7jTMLLMd8zWET8_0c4gXA%3D%3D'
driver.get(url)

# Optional: Adding some wait time for the page to fully load if needed

driver.implicitly_wait(10)  # 10 seconds 
time.sleep(20)

In [144]:
try:
    # Create a function to click away the cookies
    accept_cookies_button = driver.find_element(By.ID,"didomi-notice-agree-button")
    accept_cookies_button.click()
    print("Cookies accepted.")
except Exception as e:
    print("Cookie acceptance button not found or could not be clicked:", e)

Cookies accepted.


In [145]:
from bs4 import BeautifulSoup
import time
driver.implicitly_wait(10)

# Infinite scroll to load more content
scroll_pause_time = 5  # Adjust if necessary
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for the new page to load
    time.sleep(scroll_pause_time)
    
    # Calculate new scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # Stop if we've reached the end of the page
    last_height = new_height

# Get the final page source after all content is loaded
page_source = driver.page_source

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

In [146]:
import csv
from bs4 import BeautifulSoup

# Create empty lists to store the data
brands = []
wines = []
ratings = []
prices = []

# Find all wine entries on the page
wine_entries = soup.find_all(class_='card__card--2R5Wh wineCard__wineCardContent--3cwZt')

for entry in wine_entries:
    # Extract brand
    brand = entry.find(class_='wineInfoVintage__truncate--3QAtw')
    if brand:
        brands.append(brand.get_text(strip=True))

    # Extract wine name
    wine = entry.find(class_='wineInfoVintage__vintage--VvWlU wineInfoVintage__truncate--3QAtw') 
    if wine:
        wines.append(wine.get_text(strip=True))

    # Extract rating
    rating = entry.find(class_='vivinoRating_averageValue__uDdPM')
    if rating:
        ratings.append(rating.get_text(strip=True))

# Check for the presence of the discount first
    discount_price_div = entry.find(class_='price_strike__mOVjZ addToCart__subText--1pvFt')
    if discount_price_div:
        # If discount exist, get the original price
        discount_price_text = discount_price_div.get_text(strip=True)
        price_only = discount_price_text.split()[-1]  # Get the last part (currency + price)
        prices.append(price_only)  # Append only the discount price to the list
    
    else:  
        # Extract currency & price if present in the addToCartButton
        price_divs = entry.find_all(class_='addToCartButton__price--qJdh4')
        if price_divs:  # If primary price class exists
            for price_div in price_divs:
                currency = price_div.find('div', class_='addToCartButton__currency--2CTNX')
                price = price_div.find_all('div')[1]  # Assuming price is in the second div
                full_price = f"{currency.get_text(strip=True) if currency else ''}{price.get_text(strip=True) if price else ''}"
                prices.append(full_price)  # Save the full price
        else:  # If not present, extract price from alternative class (online verkrijgbaar vanaf...)
            alt_price_div = entry.find(class_='addToCart__subText--1pvFt addToCart__ppcPrice--ydrd5')
            if alt_price_div:
                alt_price_text = alt_price_div.get_text(strip=True)
                price_only = alt_price_text.split()[-1]  # Get the last part (currency + price)
                prices.append(price_only)  # Append only the price to the list

In [147]:
with open('sparkling_wines.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['Brand', 'Wine', 'Rating', 'Price'])

    # Write the data rows
    for brand, wine, rating, price in zip(brands, wines, ratings, prices):
        writer.writerow([
            brand,
            wine,
            rating,
            price
        ])

print("Data saved to 'sparkling_wines.csv' successfully.")

Data saved to 'sparkling_wines.csv' successfully.


## Dessert wine
Some text

In [148]:
# Dessert wines 0-500 

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Opening the 'Vivino' website
url = 'https://www.vivino.com/explore?e=eJwdi70KgCAABt_mm22Ipm9si4agKSLMLITUUPt7-6TlbjjOBhawxlHAyoelEFAv2waKdd_hyHVbeclgdJI7fFi46Kjg55dBJuO2OCl_uoQ7DSOrfMdsHfHzA9IOIGA%3D'
driver.get(url)

# Optional: Adding some wait time for the page to fully load if needed

driver.implicitly_wait(10)  # 10 seconds 
time.sleep(20)

In [149]:
try:
    # Create a function to click away the cookies
    accept_cookies_button = driver.find_element(By.ID,"didomi-notice-agree-button")
    accept_cookies_button.click()
    print("Cookies accepted.")
except Exception as e:
    print("Cookie acceptance button not found or could not be clicked:", e)

Cookies accepted.


In [150]:
from bs4 import BeautifulSoup
import time
driver.implicitly_wait(10)

# Infinite scroll to load more content
scroll_pause_time = 5  # Adjust if necessary
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for the new page to load
    time.sleep(scroll_pause_time)
    
    # Calculate new scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # Stop if we've reached the end of the page
    last_height = new_height

# Get the final page source after all content is loaded
page_source = driver.page_source

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

In [151]:
import csv
from bs4 import BeautifulSoup

# Create empty lists to store the data
brands = []
wines = []
ratings = []
prices = []

# Find all wine entries on the page
wine_entries = soup.find_all(class_='card__card--2R5Wh wineCard__wineCardContent--3cwZt')

for entry in wine_entries:
    # Extract brand
    brand = entry.find(class_='wineInfoVintage__truncate--3QAtw')
    if brand:
        brands.append(brand.get_text(strip=True))

    # Extract wine name
    wine = entry.find(class_='wineInfoVintage__vintage--VvWlU wineInfoVintage__truncate--3QAtw') 
    if wine:
        wines.append(wine.get_text(strip=True))

    # Extract rating
    rating = entry.find(class_='vivinoRating_averageValue__uDdPM')
    if rating:
        ratings.append(rating.get_text(strip=True))

# Check for the presence of the discount first
    discount_price_div = entry.find(class_='price_strike__mOVjZ addToCart__subText--1pvFt')
    if discount_price_div:
        # If discount exist, get the original price
        discount_price_text = discount_price_div.get_text(strip=True)
        price_only = discount_price_text.split()[-1]  # Get the last part (currency + price)
        prices.append(price_only)  # Append only the discount price to the list
    
    else:  
        # Extract currency & price if present in the addToCartButton
        price_divs = entry.find_all(class_='addToCartButton__price--qJdh4')
        if price_divs:  # If primary price class exists
            for price_div in price_divs:
                currency = price_div.find('div', class_='addToCartButton__currency--2CTNX')
                price = price_div.find_all('div')[1]  # Assuming price is in the second div
                full_price = f"{currency.get_text(strip=True) if currency else ''}{price.get_text(strip=True) if price else ''}"
                prices.append(full_price)  # Save the full price
        else:  # If not present, extract price from alternative class (online verkrijgbaar vanaf...)
            alt_price_div = entry.find(class_='addToCart__subText--1pvFt addToCart__ppcPrice--ydrd5')
            if alt_price_div:
                alt_price_text = alt_price_div.get_text(strip=True)
                price_only = alt_price_text.split()[-1]  # Get the last part (currency + price)
                prices.append(price_only)  # Append only the price to the list

In [152]:
with open('dessert_wines.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['Brand', 'Wine', 'Rating', 'Price'])

    # Write the data rows
    for brand, wine, rating, price in zip(brands, wines, ratings, prices):
        writer.writerow([
            brand,
            wine,
            rating,
            price
        ])

print("Data saved to 'dessert_wines.csv' successfully.")

Data saved to 'dessert_wines.csv' successfully.


## Fortified Wines

Some text

In [153]:
# Fortified wines 0-500

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Opening the 'Vivino' website
url = 'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1NTBQS6609fNRS7Z1DQ1SKwDKpqfZliUWZaaWJOao5Rel2CYWJ6vlJ1XaFhRlJqeqlZdEx9oamQB1FQMZqcVqYBIAxtccug%3D%3D'
driver.get(url)

# Optional: Adding some wait time for the page to fully load if needed

driver.implicitly_wait(10)  # 10 seconds 
time.sleep(20)

In [154]:
try:
    # Create a function to click away the cookies
    accept_cookies_button = driver.find_element(By.ID,"didomi-notice-agree-button")
    accept_cookies_button.click()
    print("Cookies accepted.")
except Exception as e:
    print("Cookie acceptance button not found or could not be clicked:", e)

Cookies accepted.


In [155]:
from bs4 import BeautifulSoup
import time
driver.implicitly_wait(10)

# Infinite scroll to load more content
scroll_pause_time = 5  # Adjust if necessary
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for the new page to load
    time.sleep(scroll_pause_time)
    
    # Calculate new scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # Stop if we've reached the end of the page
    last_height = new_height

# Get the final page source after all content is loaded
page_source = driver.page_source

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

In [156]:
import csv
from bs4 import BeautifulSoup

# Create empty lists to store the data
brands = []
wines = []
ratings = []
prices = []

# Find all wine entries on the page
wine_entries = soup.find_all(class_='card__card--2R5Wh wineCard__wineCardContent--3cwZt')

for entry in wine_entries:
    # Extract brand
    brand = entry.find(class_='wineInfoVintage__truncate--3QAtw')
    if brand:
        brands.append(brand.get_text(strip=True))

    # Extract wine name
    wine = entry.find(class_='wineInfoVintage__vintage--VvWlU wineInfoVintage__truncate--3QAtw') 
    if wine:
        wines.append(wine.get_text(strip=True))

    # Extract rating
    rating = entry.find(class_='vivinoRating_averageValue__uDdPM')
    if rating:
        ratings.append(rating.get_text(strip=True))

# Check for the presence of the discount first
    discount_price_div = entry.find(class_='price_strike__mOVjZ addToCart__subText--1pvFt')
    if discount_price_div:
        # If discount exist, get the original price
        discount_price_text = discount_price_div.get_text(strip=True)
        price_only = discount_price_text.split()[-1]  # Get the last part (currency + price)
        prices.append(price_only)  # Append only the discount price to the list
    
    else:  
        # Extract currency & price if present in the addToCartButton
        price_divs = entry.find_all(class_='addToCartButton__price--qJdh4')
        if price_divs:  # If primary price class exists
            for price_div in price_divs:
                currency = price_div.find('div', class_='addToCartButton__currency--2CTNX')
                price = price_div.find_all('div')[1]  # Assuming price is in the second div
                full_price = f"{currency.get_text(strip=True) if currency else ''}{price.get_text(strip=True) if price else ''}"
                prices.append(full_price)  # Save the full price
        else:  # If not present, extract price from alternative class (online verkrijgbaar vanaf...)
            alt_price_div = entry.find(class_='addToCart__subText--1pvFt addToCart__ppcPrice--ydrd5')
            if alt_price_div:
                alt_price_text = alt_price_div.get_text(strip=True)
                price_only = alt_price_text.split()[-1]  # Get the last part (currency + price)
                prices.append(price_only)  # Append only the price to the list

In [157]:
with open('fortified_wines.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['Brand', 'Wine', 'Rating', 'Price'])

    # Write the data rows
    for brand, wine, rating, price in zip(brands, wines, ratings, prices):
        writer.writerow([
            brand,
            wine,
            rating,
            price
        ])

print("Data saved to 'fortified_wines' successfully.")

Data saved to 'fortified_wines' successfully.
